In [4]:

# this needs to be organzied to explain the imports
from pathlib import Path
import os
from collections import defaultdict

import numpy as np
import scipy
from scipy import ndimage as ndi

import napari

# function for core algorithm
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, hole_filling
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters, img_as_float
from skimage import morphology

from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation   # function for post-processing (size filter)
from skimage.measure import label
# # package for io 
# from aicsimageio import AICSImage

from napari.utils.notebook_display import nbscreenshot



%load_ext autoreload
%autoreload 2
#import .infer_subc.base
from infer_subc.base import *

viewer = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# assumptions:   .czi "unmixed" collections.
#czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[6])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


In [ ]:

test_img_name = img_file_list[6]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


In [ ]:
data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")

out_path = data_path / "inferred_objects" 
 
default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})

################################

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 4 #2D 
med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100


Builde the segmentations in order




In [ ]:
# perform the segmentations in order to test we can build it up for one

i = 0
target_file = img_file_list[i]


img_data, meta_dict = read_input_image(target_file)
raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

raw_nuclei = img_data[0,:,:,:].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


In [ ]:

TS_GLOBAL = "Global"
TS_ADAPTIVE = "Adaptive"
TM_MANUAL = "Manual"
TM_MEASUREMENT = "Measurement"
TM_LI = "Minimum Cross-Entropy"
TM_OTSU = "Otsu"
TM_ROBUST_BACKGROUND = "Robust Background"
TM_SAUVOLA = "Sauvola"

image_data = 
mask = None
def cp_adaptive_threshold(
                image_data,
                mask,
                th_method=TM_LI, #skimage.filters.threshold_li,
                volumetric=True,
                window_size=40, 
                log_scale=False
                )


In [5]:

def MM_ac_lut(struct_img: np.ndarray) -> np.ndarray:
    """
    Allen Cell MM "Min Max" scaling from their "lookup table" 
    """
    return intensity_normalization(struct_img, scaling_param=[0])


def AC_ac_lut(structure_img0: np.ndarray) -> np.ndarray:
    """
    Allen Cell AC "Auto Contrast" scaling from their "lookup table" 
    suggest scaling parameter assuming the image is a representative example
    of this cell structure
    """
    m, s = norm.fit(structure_img0.flat)    
    p99 = np.percentile(structure_img0, 99.99)
    pmin = structure_img0.min()
    pmax = structure_img0.max()    
    up_ratio = 0
    for up_i in np.arange(0.5, 1000, 0.5):
        if m + s * up_i > p99:
            if m + s * up_i > pmax:
                up_ratio = up_i - 0.5
            else:
                up_ratio = up_i
            break

    low_ratio = 0
    for low_i in np.arange(0.5, 1000, 0.5):
        if m - s * low_i < pmin:
            low_ratio = low_i - 0.5
            break

    print(f"parameter for normalization is [{low_ratio}, {up_ratio}]")
    struct_img = intensity_normalization(structure_img0, scaling_param=[low_ratio, up_ratio])
    return struct_img



def G2_ac_lut(struct_img: np.ndarray, sigma: float) -> np.ndarray:
    """
    Allen Cell G2 "slice_by_slice" smoothing   from their "lookup table" 
    TODO: use partials
    """
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=sigma,
                                                                                                        truncate_range = 3.0
                                                                                                    )
    return struct_img

def G3_ac_lut(struct_img: np.ndarray, sigma: float) -> np.ndarray:
    """
    Allen Cell G3 "3D" smoothing   from their "lookup table" 
    TODO: use partials
    """
    struct_img = image_smoothing_gaussian_3d(   struct_img,
                                                                                                        sigma=sigma,
                                                                                                        truncate_range = 3.0
                                                                                                    )
    return struct_img

def ES_ac_lut(   struct_img: np.ndarray ) -> np.ndarray:
    """
    Allen Cell ES "Edge preserving" smoothing   from their "lookup table" 
    TODO: use partials
    """
    return edge_preserving_smoothing_3d(struct_img) 

def TT_ac_lut(   struct_img: np.ndarray ) -> np.ndarray:
    """
    Allen Cell TT "Topology-preserving Thinning" smoothing   from their "lookup table" 
    TODO: use partials
    """
    return topology_preserving_thinning( struct_img ) 
 
def S2_ac_lut(   struct_img: np.ndarray, s2_param:list ) -> np.ndarray:
    """
    Allen Cell S2 "Edge preserving" smoothing   from their "lookup table" 
    TODO: use partials
    """
    bw_spot = dot_2d_slice_by_slice_wrapper(struct_img, s2_param)
    return bw_spot

def S3_ac_lut(   struct_img: np.ndarray ) -> np.ndarray:
    """
    Allen Cell S3 "Edge preserving" smoothing   from their "lookup table" 
    TODO: use partials
    """
    bw_spot = dot_3d_slice_by_slice_wrapper(struct_img, s2_param)
    return bw_spot



##########################
# 2a.  infer_SOMA1
##########################
def _infer_SOMA1(struct_img: np.ndarray, NU_label: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_label: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice(  struct_img,
                                                                            size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # # non-Linear processing
    # log_img, d = log_transform( struct_img ) 
    # log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    # struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img

    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    # struct_obj, _bw_low_level = MO(struct_img, 
    #                                             global_thresh_method='ave', 
    #                                             object_minArea=low_level_min_size, 
    #                                             extra_criteria=True,
    #                                             local_adjust= local_adjust, 
    #                                             return_object=True,
    #                                             dilate=True)

    threshold_value_log = threshold_multiotsu_log(struct_img)
    print(f"threshold is {threshold_value_log.shape}")
    threshold_value_log = threshold_otsu_log(struct_img)

    # thresh_factor = 0.9 #from cellProfiler
    # thresh_min = 0.1
    # thresh_max = 1.0
    # threshold = min( max(threshold_value_log*thresh_factor, thresh_min), thresh_max )
    struct_obj = struct_img > threshold_value_log

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label > 0),
                )

                                

    retval = (struct_obj,  labels_out, out_p)
    return retval
    
    # ###################
    # # POST- POST_PROCESSING
    # ###################
    # # keep the "SOMA" label which contains the highest total signal
    # all_labels = np.unique(labels_out)[1:]

    # total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # # combine NU and "labels" to make a SOMA
    # keep_label = all_labels[np.argmax(total_signal)]

    # # now use all the NU labels which AREN't keep_label and add to mask and re-label
    # masked_composite_soma = struct_img.copy()
    # new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # # "Masked Object Thresholding" - 3D
    # masked_composite_soma[new_NU_mask] = 0
    # struct_obj, _bw_low_level = MO(masked_composite_soma, 
    #                                             global_thresh_method='ave', 
    #                                             object_minArea=low_level_min_size, 
    #                                             extra_criteria=True,
    #                                             local_adjust= local_adjust, 
    #                                             return_object=True,
    #                                             dilate=True)

    # struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    # struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
    #                                                         min_size= small_object_width**3, 
    #                                                         method = "slice_by_slice" ,
    #                                                         connectivity=1)
    # masked_labels_out = watershed(
    #             connectivity=np.ones((3, 3,3), bool),
    #             image=1. - struct_img,
    #             markers=NU_label,
    #             mask= np.logical_or(struct_obj, NU_label == keep_label),
    #             )
                

    # retval = (struct_obj,  masked_labels_out, out_p)
    # return retval



# ###  Discard edges algo... tricky

# labels_in = objects.unedited_segmented.copy()
#         labels_touching_edge = numpy.hstack(
#             (labels_in[0, :], labels_in[-1, :], labels_in[:, 0], labels_in[:, -1])
#         )
#         labels_touching_edge = numpy.unique(labels_touching_edge)
#         is_touching = numpy.zeros(numpy.max(labels_in) + 1, bool)
#         is_touching[labels_touching_edge] = True
#         is_touching = is_touching[labels_in]

#         labels_in[(~is_touching) & (objects.segmented == 0)] = 0
#         #
#         # Stretch the input labels to match the image size. If there's no
#         # label matrix, then there's no label in that area.
#         #
#         if tuple(labels_in.shape) != tuple(img.shape):
#             tmp = numpy.zeros(img.shape, labels_in.dtype)
#             i_max = min(img.shape[0], labels_in.shape[0])
#             j_max = min(img.shape[1], labels_in.shape[1])
#             tmp[:i_max, :j_max] = labels_in[:i_max, :j_max]
#             labels_in = tmp

#      if self.wants_discard_edge:
#             lookup = scipy.ndimage.maximum(
#                 segmented_out,
#                 objects.segmented,
#                 list(range(numpy.max(objects.segmented) + 1)),
#             )
#             lookup = centrosome.cpmorphology.fixup_scipy_ndimage_result(lookup)
#             lookup[0] = 0
#             lookup[lookup != 0] = numpy.arange(numpy.sum(lookup != 0)) + 1
#             segmented_labels = lookup[objects.segmented]
#             segmented_out = lookup[segmented_out]

        


In [6]:


##########################
# 2b.  infer_SOMA2
########################### copy this to base.py for easy import
def _infer_SOMA2(struct_img: np.ndarray, NU_label: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_label: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param


    # 2D smoothing
    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 9   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 3.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    #    edges = filters.scharr(struct_img)
    # struct_img, d = log_transform( struct_img ) 
    # struct_img = intensity_normalization(  struct_img,  scaling_param=[0] )
    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.25
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################
    # 3D cleaning

    hole_max = 80  
    # discount z direction
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(struct_img)),
                                                markers=NU_label,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_label > 0),
                                                )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    # 3D cleaning
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "3D", #"slice_by_slice" 
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=np.abs(ndi.sobel(struct_img)),
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)
    return retval

In [7]:


##########################
# 2c.  infer_SOMA3
##########################
def _infer_SOMA3(struct_img, NU_label,  in_params) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_label: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice( 
                                                                    struct_img,
                                                                    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    log_img, d = log_transform( struct_img ) 
    struct_img = intensity_normalization(  log_img + filters.scharr(log_img) ,  scaling_param=[0] )  


    ###################
    # CORE_PROCESSING
    ###################
    # "Masked Object Thresholding" - 3D
    local_adjust = 0.5
    low_level_min_size = 100
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)
    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 
    ###################
    # POST_PROCESSING
    ###################
    # 2D cleaning
    hole_max = 100  
    # discount z direction
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 30
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                                                image=np.abs(ndi.sobel(struct_img)),  #either log_img or struct_img seem to work, but more spurious labeling to fix in post-post for struct_img
                                                markers=NU_label,
                                                connectivity=np.ones((3, 3, 3), bool),
                                                mask= np.logical_or(struct_obj, NU_label > 0),
                                                )

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    # 2D cleaning
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=np.abs(ndi.sobel(struct_img)),  #either log_img or struct_img seem to work, but more spurious labeling to fix in post-post for struct_img
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label == keep_label),
                )
                
    retval = (struct_obj,  masked_labels_out, out_p)
    return retval


In [8]:
data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")

out_path = data_path / "inferred_objects" 
czi_img_folder = data_path/"raw"
list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
 
default_params = defaultdict(str, **{
    #"intensity_norm_param" : [0.5, 15]
    "intensity_norm_param" : [0],
    "gaussian_smoothing_sigma" : 1.34,
    "gaussian_smoothing_truncate_range" : 3.0,
    "dot_2d_sigma" : 2,
    "dot_2d_sigma_extra" : 1,
    "dot_2d_cutoff" : 0.025,
    "min_area" : 10,
    "low_level_min_size" :  100,
    "median_filter_size" : 10
})


################################

idx = 4
target_file = img_file_list[idx]


img_data, meta_dict = read_input_image(target_file)
raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

# calculate a filter dimension for median filtering which considers the difference in scale of Z
z_factor = scale[0]//scale[1]
med_filter_size = 15 #2D 
med_filter_size_3D = (np.round(med_filter_size*z_factor),med_filter_size,med_filter_size)  # set the scale for a typical median filter
print(f"median filtering scale is ~ : { [x*y for x,y in zip(scale,med_filter_size_3D)]}")

default_params['z_factor'] = z_factor
default_params['scale'] = scale
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


median filtering scale is ~ : [60.947535214869504, 1.1980747777255978, 1.1980747777255978]


In [9]:

raw_nuclei = img_data[0,:,:,:].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 



intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [10]:

###################
# SOMA1
###################
raw_soma1 = (4. * img_data[1,:,:,:].copy() + 
                            1. * img_data[5,:,:,:].copy() + 
                            1. * img_data[7,:,:,:].copy() )

struct_img = raw_soma1.copy()

NU_eroded = morphology.binary_erosion(NU_object)  

mask = NU_label>0
struct_img[mask] = 0

###################
# PRE_PROCESSING
###################                         
scaling_param =  [0]   
struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)

# # Linear-ish processing
# med_filter_size = 15   
struct_img = ndi.median_filter(struct_img,    size=med_filter_size  )
# struct_img = median_filter_slice_by_slice(  struct_img,
#                                                                         size=15  )
out_p["median_filter_size"] = med_filter_size 
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
# struct_img1 = image_smoothing_gaussian_slice_by_slice(   struct_img,
#                                                                                                     sigma=gaussian_smoothing_sigma,
#                                                                                                     truncate_range = gaussian_smoothing_truncate_range
#                                                                                                 )
struct_img = image_smoothing_gaussian_3d(   struct_img,
                                                                                                    sigma=gaussian_smoothing_sigma,
                                                                                                    truncate_range = gaussian_smoothing_truncate_range
                                                                                                )
                                                                                                
# # non-Linear processing
# log_img, d = log_transform( struct_img ) 
# log_img = intensity_normalization(  log_img,  scaling_param=[0] )

# struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img
sobel_image = np.abs(ndi.sobel(struct_img))

threshold_value_log = threshold_multiotsu_log(struct_img)
print(f"multiotsu threshold is {threshold_value_log}")
threshold_value_log = threshold_otsu_log(struct_img)

# thresh_factor = 0.9 #from cellProfiler
# thresh_min = 0.1
# thresh_max = 1.0
# threshold = min( max(threshold_value_log*thresh_factor, thresh_min), thresh_max )
bw1 = struct_img > threshold_value_log

#   "Masked Object Thresholding" - 3D
local_adjust = 0.7
low_level_min_size = 30
bw2, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)


intensity normalization: min-max normalization with NO absoluteintensity upper bound
multiotsu threshold is [0.00278943 0.0097979 ]


In [ ]:

###################
# POST_PROCESSING
###################

# 2D 
hole_max = 80  
struct_obj2 = hole_filling(bw2, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
out_p['hole_max'] = hole_max

small_object_width = 30
struct_obj2 = size_filter(struct_obj2, # wrapper to remove_small_objects which can do slice by slice
                                                        min_size= small_object_width**2, 
                                                        method = "slice_by_slice" ,
                                                        connectivity=1)
out_p['small_object_width'] = small_object_width

# 2D 
hole_max = 80  
struct_obj1 = hole_filling(bw1, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
out_p['hole_max'] = hole_max

small_object_width = 30
struct_obj1 = size_filter(struct_obj1, # wrapper to remove_small_objects which can do slice by slice
                                                        min_size= small_object_width**2, 
                                                        method = "slice_by_slice" ,
                                                        connectivity=1)
out_p['small_object_width'] = small_object_width


In [13]:

labels_out1a = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=1. - struct_img,
            markers=NU_label,
            mask= struct_obj2#np.logical_and(struct_obj),
            )

labels_out1b = watershed(
            connectivity=np.ones((3, 3,3), bool),
            image=sobel_image,
            markers=NU_label,
            mask= struct_obj2 #np.logical_and(struct_obj, ~mask),
            )
             
#SO_object1, SO_label1, out_p =  _infer_SOMA1(raw_soma1.copy(), NU_label, out_p) 


In [14]:

marker_locations = ndi.center_of_mass(NU_label, NU_label,np.unique(NU_label)[1:])

markers = np.zeros(NU_label.shape, dtype=np.uint32)
marker_indices = tuple(np.round(marker_locations).astype(int).T)
markers[marker_indices] = np.arange(len(marker_locations)) + 1
markers_big = morphology.dilation(markers, morphology.ball(5))



In [15]:
markers_big.sum()

23174

In [16]:

segmented = watershed(
    sobel_image,
    markers_big,
    mask=struct_obj2,
)

In [23]:

viewer.dims.ndisplay = 2

cmap = 'green'
viewer.add_image(
    struct_obj2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    markers_big,
    scale=scale,
)

viewer.add_labels(
    segmented,
    scale=scale,
)
viewer.add_image(
    struct_img,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)


NameError: name 'add_image' is not defined

In [24]:

viewer.add_image(
    struct_obj1,
    scale=scale,
    colormap='red', 
    blending='additive'
)
    


<Image layer 'struct_obj1' at 0x17195c940>

In [21]:
viewer.add_labels(
    segmented,
    scale=scale,
)


<Labels layer 'segmented [1]' at 0x14ea71be0>

In [22]:

viewer.add_labels(
    labels_out1a,
    scale=scale,
)

viewer.add_labels(
    labels_out1b,
    scale=scale,
)


<Labels layer 'labels_out1b' at 0x16f094dc0>

In [34]:

# viewer.add_labels(
#     labels_out1a,
#     scale=scale,
#     blending='additive'
# )

# viewer.add_labels(
#     labels_out1b,
#     scale=scale,
#     blending='additive'
# )
cmap = 'green'
viewer.add_image(
    struct_img1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)
viewer.add_image(
    sobel_image1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    1-struct_img1,
    name='neg_struct_image',
    scale=scale,
    colormap=cmap, 
    blending='additive'
)
viewer.add_image(
    bw1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

cmap = 'red'
viewer.add_image(
    struct_img2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)
viewer.add_image(
    sobel_image2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    1-struct_img2,
    name='neg_struct_image2',
    scale=scale,
    colormap=cmap, 
    blending='additive'
)
viewer.add_image(
    bw2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)


<Image layer 'bw2' at 0x14c5b8d90>

25-Aug-22 21:25:39 - vispy    - WARNING  - Error drawing visual <Volume at 0x1697e00d0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 903, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 218, in on_draw
    self._draw_scene()
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 277, in _draw_scene
    self.draw_vis

In [30]:
viewer = napari.view_image(
    raw_nuclei,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.scale_bar.visible = True

viewer.add_labels(
    NU_label,
    scale=scale,
    blending='additive'
)

viewer.add_labels(
    SO_label1,
    scale=scale,
    blending='additive'
)
cmap = 'green'
viewer.add_image(
    raw_soma1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)
viewer.add_image(
    1-SO_object1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)


<Image layer 'Image' at 0x1697fd0a0>

In [ ]:


# viewer.add_image(
#     NU_object,
#     scale=scale,
#     blending='additive'
# )

viewer.add_image(
    SO_object1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

cmap = 'red'
viewer.add_image(
    raw_soma2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    SO_object2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

cmap = 'magenta'
viewer.add_image(
    raw_soma3,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    SO_object3,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)


In [31]:

viewer.dims.ndisplay = 3
viewer.camera.angles = (-30, 25, 120)


25-Aug-22 21:05:24 - vispy    - WARNING  - Error drawing visual <Volume at 0x1697e00d0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 903, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 218, in on_draw
    self._draw_scene()
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 277, in _draw_scene
    self.draw_vis

In [17]:

###################
# SOMA2
###################
composite_channels = [1,4,5,7]
raw_soma_ = img_data[composite_channels,:,:,:].copy()
raw_soma_ =   raw_soma_  / raw_soma_.max()
raw_soma2 = raw_soma_.sum(axis=0)
SO_object2, SO_label2, out_p =  _infer_SOMA2(raw_soma2.copy(), NU_label, out_p) 

###################
# SOMA3
###################
composite_channels = [1]
raw_soma3 = intensity_normalization(  img_data[1,:,:,:].copy(), scaling_param=[0] )

#####################

SO_object3, SO_labe3l, out_p =  _infer_SOMA3(raw_soma3.copy(), NU_label, out_p) 



intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


KeyboardInterrupt: 

In [18]:

ndi.extrema(raw_soma1),ndi.extrema(raw_soma2), ndi.extrema(raw_soma3)
#raw_soma1.max(), raw_soma1.min()


((32.0, 270380.0, (1, 759, 293), (9, 418, 277)),
 (0.0004882887006942855, 1.4409857328145264, (1, 759, 293), (10, 62, 751)),
 (1.5259021896694095e-13, 1.0, (0, 0, 5), (9, 418, 277)))

In [19]:
viewer = napari.view_image(
    raw_nuclei,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.scale_bar.visible = True

viewer.add_labels(
    NU_label,
    scale=scale,
    blending='additive'
)

# viewer.add_image(
#     NU_object,
#     scale=scale,
#     blending='additive'
# )
cmap = 'green'
viewer.add_image(
    raw_soma1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    SO_object1,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

cmap = 'red'
viewer.add_image(
    raw_soma2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    SO_object2,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

cmap = 'magenta'
viewer.add_image(
    raw_soma3,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.add_image(
    SO_object3,
    scale=scale,
    colormap=cmap, 
    blending='additive'
)

viewer.dims.ndisplay = 3
viewer.camera.angles = (-30, 25, 120)


25-Aug-22 15:19:24 - vispy    - WARNING  - Error drawing visual <Volume at 0x16c51c7f0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 903, in paintGL
    self._vispy_canvas.events.draw(region=None)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 218, in on_draw
    self._draw_scene()
  File "/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/vispy/scene/canvas.py", line 277, in _draw_scene
    self.draw_vis

In [20]:
viewer.dims.ndisplay = 2


In [ ]:



###################
# PRE_PROCESSING
###################                         

#TODO: replace params below with the input params
scaling_param =  [0]   
struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
out_p["intensity_norm_param"] = scaling_param

# make a copy for post-post processing
scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice(  struct_img,
                                                                            size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # non-Linear processing
    log_img, d = log_transform( struct_img ) 
    log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img

    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label > 0),
                )
    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)


#SO_object, SO_label, out_p =  _infer_SOMA1(struct_img.copy(), NU_label, out_p) 

In [ ]:



###################
# PRE_PROCESSING
###################                         

#TODO: replace params below with the input params
scaling_param =  [0]   
struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
out_p["intensity_norm_param"] = scaling_param

# make a copy for post-post processing
scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice(  struct_img,
                                                                            size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # non-Linear processing
    log_img, d = log_transform( struct_img ) 
    log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img

    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label > 0),
                )
    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)


#SO_object, SO_label, out_p =  _infer_SOMA1(struct_img.copy(), NU_label, out_p) 

In [ ]:
##########################
# 2a.  infer_SOMA1
##########################
def _infer_SOMA1(struct_img: np.ndarray, NU_label: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_label: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = median_filter_slice_by_slice(  struct_img,
                                                                            size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # non-Linear processing
    log_img, d = log_transform( struct_img ) 
    log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img

    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    out_p['hole_max'] = hole_max

    small_object_width = 35
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    out_p['small_object_width'] = small_object_width

    labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label > 0),
                )
    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    # now use all the NU labels which AREN't keep_label and add to mask and re-label
    masked_composite_soma = struct_img.copy()
    new_NU_mask = np.logical_and( NU_label !=0 ,NU_label != keep_label)

    # "Masked Object Thresholding" - 3D
    masked_composite_soma[new_NU_mask] = 0
    struct_obj, _bw_low_level = MO(masked_composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = size_filter(struct_obj, # wrapper to remove_small_objects which can do slice by slice
                                                            min_size= small_object_width**3, 
                                                            method = "slice_by_slice" ,
                                                            connectivity=1)
    masked_labels_out = watershed(
                connectivity=np.ones((3, 3,3), bool),
                image=1. - struct_img,
                markers=NU_label,
                mask= np.logical_or(struct_obj, NU_label == keep_label),
                )
                

    retval = (struct_obj,  masked_labels_out, out_p)
    return retval


